In [2]:
EPS = 1e-6
#slightly modified
def get_IoU(outputs, labels):
    outputs = outputs.int()
    labels = labels.int()
    # Taken from: https://www.kaggle.com/iezepov/fast-iou-scoring-metric-in-pytorch-and-numpy
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))  # Will be zero if both are 0

    iou = (intersection + EPS) / (union + EPS)  # We smooth our devision to avoid 0/0

    # thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    # return thresholded.mean()  # Or thresholded.mean() if you are interested in average across the batch
    return iou.mean()

In [10]:
import torch
true = torch.tensor([[
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
]])
pred = torch.tensor([[
    [0.1, 0, 0],
    [0.1, 0, 0],
    [0.9, 0, 0],
]])
print(true.shape, pred.shape)
print(get_IoU(pred, true))

torch.Size([1, 3, 3]) torch.Size([1, 3, 3])
tensor(3.3333e-07)


In [9]:
# computes confusion matrix
def _fast_hist(true, pred, num_classes):
    mask = (true >= 0) & (true < num_classes)
    hist = torch.bincount(
        num_classes * true[mask] + pred[mask],
        minlength=num_classes ** 2,
    ).reshape(num_classes, num_classes).float()
    return hist
def nanmean(v, *args, inplace=False, **kwargs):
    if not inplace:
        v = v.clone()
    is_nan = torch.isnan(v)
    v[is_nan] = 0
    return v.sum(*args, **kwargs) / (~is_nan).float().sum(*args, **kwargs)
# computes IoU based on confusion matrix
def jaccard_index(hist):
    """Computes the Jaccard index, a.k.a the Intersection over Union (IoU).
    Args:
        hist: confusion matrix.
    Returns:
        avg_jacc: the average per-class jaccard index.
    """
    A_inter_B = torch.diag(hist)
    A = hist.sum(dim=1)
    B = hist.sum(dim=0)
    jaccard = A_inter_B / (A + B - A_inter_B + EPS)
    print(jaccard)
    avg_jacc = nanmean(jaccard) #the mean of jaccard without NaNs
    return avg_jacc, jaccard

torch.Size([1, 3, 3]) torch.Size([1, 3, 3])
tensor(1.)


In [42]:
import torch
true = torch.tensor([[
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
]])
pred = torch.tensor([[
    [0.8, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
]])
print(true.shape, pred.shape)
print(get_IoU(pred, true))
hist = _fast_hist(pred, true, num_classes=2)
print(jaccard_index(hist))

torch.Size([1, 3, 3]) torch.Size([1, 3, 3])
tensor(0.6667)


RuntimeError: "bincount_cpu" not implemented for 'Float'

In [12]:
from sklearn.metrics import f1_score, roc_auc_score

In [37]:
import numpy as np
predict=np.array([[0.1, 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])
label=np.array([[1, 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [34]:
l=label.flatten()
p=predict.flatten()
print(l,p)

[1. 1. 1. 0. 0. 0. 0. 0. 0.] [0.8 1.  1.  0.  0.  0.  0.  0.  0. ]


In [35]:
f1_score(predict.ravel(),label.ravel())

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [39]:
roc_auc_score(l,p)

1.0

In [47]:
 from sklearn.metrics import confusion_matrix  
 import numpy as np

 def compute_iou(y_pred, y_true):
     # ytrue, ypred is a flatten vector
     y_pred = y_pred.flatten()
     y_true = y_true.flatten()
     current = confusion_matrix(y_true, y_pred, labels=[0, 1])
     # compute mean iou
     intersection = np.diag(current)
     ground_truth_set = current.sum(axis=1)
     predicted_set = current.sum(axis=0)
     union = ground_truth_set + predicted_set - intersection
     IoU = intersection / union.astype(np.float32)
     return np.mean(IoU)

In [48]:
compute_iou(predict,label)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [22]:
import numpy as np
predict=np.array([[1, 1., 1.],
       [0., 0., 0.1],
       [0., 0., 0.]])
label=np.array([[1, 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [23]:
pp=predict>0.01
pp

array([[ True,  True,  True],
       [False, False,  True],
       [False, False, False]])

In [30]:
ll=label>0.999
ll

array([[ True,  True,  True],
       [False, False, False],
       [False, False, False]])

In [24]:
from sklearn.metrics import confusion_matrix

In [33]:
cm=confusion_matrix(ll.ravel(),pp.ravel())
tn, fp, fn, tp = cm.ravel()
print(tp,fp,tn,fn)

3 1 5 0


In [18]:
import numpy as np

A = np.array([[1,4],[2,5],[3,6]])
B = np.array([[1,4],[3,6],[7,8]])

nrows, ncols = A.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [A.dtype]}

C = np.intersect1d(A.view(dtype), B.view(dtype))

# This last bit is optional if you're okay with "C" being a structured array...
C = C.view(A.dtype).reshape(-1, ncols)

In [19]:
C

array([[1, 4],
       [3, 6]])